In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# zip 파일을 위한 모듈
import os
import io
import zipfile
from io import BytesIO
from zipfile import ZipFile

In [2]:
# 데이터 로드
# 특정 파일만 압축풀기
with ZipFile('../data/open.zip','r') as zipObj:
    list_filenames = zipObj.namelist()
    for filename in list_filenames:
        if filename.endswith('.csv'):
            zipRead = zipObj.read(filename)
            globals()[f'{filename[:-4]}'] = pd.read_csv(BytesIO(zipRead))
# train.csv >> train , test.csv >> test

<hr/>

In [13]:
# 날짜 데이터 전처리
# datetime 컬럼 처리
train['ATA'] = pd.to_datetime(train['ATA'])
test['ATA'] = pd.to_datetime(test['ATA'])

# datetime을 여러 파생 변수로 변환
for df in [train, test]:
    df['year'] = df['ATA'].dt.year
    df['month'] = df['ATA'].dt.month
    df['day'] = df['ATA'].dt.day
    df['hour'] = df['ATA'].dt.hour

# datetime 컬럼 제거
train.drop(columns='ATA', inplace=True)
test.drop(columns='ATA', inplace=True)

# ATA_LT 컬럼도 불필요
train.drop(columns='ATA_LT', inplace=True)
test.drop(columns='ATA_LT', inplace=True)

In [5]:
# 결측치 처리
train.isnull().sum()

SAMPLE_ID                  0
ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    1
BUILT                      0
DEADWEIGHT                 0
DEPTH                      1
DRAUGHT                    1
GT                         0
LENGTH                     1
SHIPMANAGER                0
FLAG                       0
U_WIND                163688
V_WIND                163688
AIR_TEMPERATURE       164630
BN                    163688
PORT_SIZE                  0
CI_HOUR                    0
year                       0
month                      0
day                        0
hour                       0
dtype: int64

In [6]:
train[train['BREADTH'].isnull()].isnull().sum()
# 'BREADTH' 가 nan인 행이 1개 있고, 그 행은 depth, draught, length가 모두 nan이다.

SAMPLE_ID             0
ARI_CO                0
ARI_PO                0
SHIP_TYPE_CATEGORY    0
DIST                  0
ID                    0
BREADTH               1
BUILT                 0
DEADWEIGHT            0
DEPTH                 1
DRAUGHT               1
GT                    0
LENGTH                1
SHIPMANAGER           0
FLAG                  0
U_WIND                0
V_WIND                0
AIR_TEMPERATURE       0
BN                    0
PORT_SIZE             0
CI_HOUR               0
year                  0
month                 0
day                   0
hour                  0
dtype: int64

In [7]:
# 같은 종류의 SHIP_TYPE으로 결측치 처리
Cargo_ships=train[train['SHIP_TYPE_CATEGORY']==train[train['BREADTH'].isnull()]['SHIP_TYPE_CATEGORY'].iloc[0]]

In [8]:
# 결측치 채우기
idx=train[train['BREADTH'].isnull()].index[0]
train.loc[idx,['BREADTH','DEPTH','DRAUGHT','LENGTH']] = Cargo_ships[['BREADTH','DEPTH','DRAUGHT','LENGTH']].mean()

In [9]:
train.loc[idx] #완료

SAMPLE_ID             TRAIN_356484
ARI_CO                          TW
ARI_PO                        JWI3
SHIP_TYPE_CATEGORY           Cargo
DIST                           0.0
ID                         A457875
BREADTH                  17.286295
BUILT                           28
DEADWEIGHT                    1500
DEPTH                    10.422017
DRAUGHT                   6.659886
GT                             500
LENGTH                  107.555142
SHIPMANAGER                 GVPJ58
FLAG                         Japan
U_WIND                       -0.34
V_WIND                       -1.91
AIR_TEMPERATURE               25.3
BN                        1.752807
PORT_SIZE                  0.00099
CI_HOUR                        0.0
year                          2022
month                           11
day                              8
hour                             4
Name: 356484, dtype: object

<hr/>

In [10]:
train.isnull().sum()
# ARI와 ATA가 같거나 비슷한 데이터의 평균으로 대체

SAMPLE_ID                  0
ARI_CO                     0
ARI_PO                     0
SHIP_TYPE_CATEGORY         0
DIST                       0
ID                         0
BREADTH                    0
BUILT                      0
DEADWEIGHT                 0
DEPTH                      0
DRAUGHT                    0
GT                         0
LENGTH                     0
SHIPMANAGER                0
FLAG                       0
U_WIND                163688
V_WIND                163688
AIR_TEMPERATURE       164630
BN                    163688
PORT_SIZE                  0
CI_HOUR                    0
year                       0
month                      0
day                        0
hour                       0
dtype: int64